<a href="https://colab.research.google.com/github/TheDarKnight50/SOC---LLM/blob/main/Lectures/Lec_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
!wget https://raw.githubusercontent.com/karpathy/makemore/master/names.txt

--2024-06-26 13:24:58--  https://raw.githubusercontent.com/karpathy/makemore/master/names.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 228145 (223K) [text/plain]
Saving to: ‘names.txt’

names.txt           100%[===================>] 222.80K  --.-KB/s    in 0.1s    

2024-06-26 13:24:58 (1.85 MB/s) - ‘names.txt’ saved [228145/228145]



In [3]:
words = open('/content/names.txt', 'r').read().splitlines()
words[:10]

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

In [4]:
len(words)

32033

In [5]:
chars = sorted(list(set(''.join(words))))
stoi = {s : i + 1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i : s for s, i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [6]:
# Building the dataset

def build_dataset(words) :
  block_size = 3
  X, Y = [], []
  for w in words:
    # print(w)
    context = [0] * block_size # Build a list corresponding to : [0, 0, 0, ...]

    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      # print(''.join(itos[i] for i in context), '--->', itos[ix])
      context = context[1:] + [ix]  # Crop from first letter and slide

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr,  Ytr  = build_dataset(words[:n1])     # 80%
Xdev, Ydev = build_dataset(words[n1:n2])   # 10%
Xte,  Yte  = build_dataset(words[n2:])     # 10%

In [7]:
C = torch.randn(27, 2)

In [8]:
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

In [9]:
W1 = torch.randn((6, 100))
b1 = torch.randn(100)

In [10]:
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
h

tensor([[ 0.1821,  0.9999, -0.1277,  ..., -0.9989, -0.1337, -0.8470],
        [ 0.0832,  0.9999,  0.4440,  ..., -0.9964, -0.0904, -0.9091],
        [ 0.8431,  0.9992, -0.9901,  ..., -0.9998,  0.0938, -0.1653],
        ...,
        [ 0.6218,  0.9958, -0.8534,  ..., -0.9999, -0.8505, -0.8600],
        [-0.2991,  1.0000,  1.0000,  ..., -0.9997, -0.1989, -0.9455],
        [ 0.7742,  1.0000, -0.7255,  ..., -0.9956,  0.9328, -0.9997]])

In [11]:
W2 = torch.randn((100, 27))
b2 = torch.randn(27)

In [33]:
logits = h @ W2 + b2
counts = logits.exp()
prob = counts / counts.sum(1, keepdims = True)
prob.shape

torch.Size([32, 27])

In [34]:
loss = -prob[torch.arange(32), Y].log().mean()
loss

IndexError: shape mismatch: indexing tensors could not be broadcast together with shapes [32], [228146]

In [ ]:
# -------- summary of above : ----------

In [22]:
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27, 10), generator = g)

W1 = torch.randn((30, 200), generator = g)
b1 = torch.randn(200, generator = g)

W2 = torch.randn((200, 27), generator = g)
b2 = torch.randn(27, generator = g)

parameters = [C, W1, b1, W2 ,b2]

In [25]:
sum(p.nelement() for p in parameters)

11897

In [26]:
emb = C[Xtr]
h = torch.tanh(emb.view(-1, 30) @ W1 + b1)

logits = h @ W2 + b2
#counts = logits.exp()
#prob = counts / counts.sum(1, keepdims = True)
#prob.shape

loss = F.cross_entropy(logits, Ytr) # Inbuilt function to do the same thing :)

In [27]:
for p in parameters : p.requires_grad = True

In [31]:
for i in range(10000) :
  # Construct mini-batches :
  ix = torch.randint(0, Xtr.shape[0], (32,))

  # Forward pass :
  emb = C[Xtr[ix]]
  h = torch.tanh(emb.view(-1, 30) @ W1 + b1)

  logits = h @ W2 + b2

  loss = F.cross_entropy(logits, Ytr[ix]) # Inbuilt function to do the same thing :)

  # Backward pass :
  for p in parameters :
    p.grad = None

  loss.backward()

  # Update :
  lr = - 0.01
  for p in parameters :
    p.data += lr * p.grad

print(loss.item())

2.1206839084625244


In [21]:
emb = C[Xdev]
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)

logits = h @ W2 + b2

loss = F.cross_entropy(logits, Ydev)
loss

tensor(2.6440, grad_fn=<NllLossBackward0>)